In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm


pd.options.display.max_rows = 35 
pd.options.display.max_columns = None

In [3]:
col_ignore = ['scheme_name', 'wpt_name', 'source_class', 'quality_group', 'quantity_group', 'recorded_by', 
             'extraction_type_group', 'extraction_type_class','management_group', 'payment_type', 'region_code',
              'district_code', 'subvillage', 'scheme_management', 'source_type', 'waterpoint_type_group', 
              'num_private', 'funder']

water_values = pd.read_csv('../../data/raw/WaterValuesOriginal.csv')
# water_values = water_values[[i for i in water_values.columns if i not in col_ignore]]
water_labels = pd.read_csv('../../data/raw/WaterLabelsOriginal.csv')

water_labels['target'] = water_labels.status_group.values  #change target column
water_labels.drop('status_group', axis = 1, inplace = True) # drop the original column

water_values = pd.merge(water_values, water_labels, on = 'id') #merge target and features

water_values['date_recorded_date'] = pd.to_datetime(water_values.date_recorded)
water_values['year_recorded'] = pd.DatetimeIndex(water_values.date_recorded_date).year
print(water_values.shape)
print(len(water_values[water_values.year_recorded < 2011]))

#target distribution is roughly the same as the population distribution so we decided to drop them because there
#are not many entried and because the recorded was earlier than construction (all in 2004)

water_values = water_values[water_values.year_recorded >= 2011].reset_index(drop = True)



FileNotFoundError: [Errno 2] File ../../data/raw/WaterValuesOriginal.csv does not exist: '../../data/raw/WaterValuesOriginal.csv'

In [ ]:
water_values.quantity.unique()# water_values.payment_type.unique()
# water_values.payment_type.unique()
# water_values.source_class.unique()
#water_values[water_values.source != 'unknown']

In [1]:
for i in water_values.water_quality.unique(): 
    print(i)
    print(water_values[water_values.water_quality == i].target.value_counts(), '\n')

NameError: name 'water_values' is not defined

In [42]:
# print('Source_type')
# for i in water_values.source_type.unique(): 
#     print(i)
#     if 'unknown' in i: 
#         print(water_values[source_type.source==i].target.value_counts(), '\n')

# print('quality_group')
# for i in water_values.quality_group.unique(): 
#     if 'unknown' in i: 
#         print(water_values[quality_group.water_quality==i].target.value_counts(), '\n')
        
# print('quantity_group')
# for i in water_values.quantity_group.unique(): 
#     if 'unknown' in i: 
#         print(water_values[water_values.quantity_group==i].target.value_counts(), '\n')
        
# print('Payment_type')
# for i in water_values.payment_type.unique(): 
#     if 'unknown' in i: 
#         print(water_values[water_values==i].target.value_counts(), '\n')

In [10]:
print('Source')
for i in water_values.source.unique(): 
    if 'unknown' in i: 
        print(water_values[water_values.source==i].target.value_counts(), '\n')

print('Water_Quality')
for i in water_values.water_quality.unique(): 
    if 'unknown' in i: 
        print(water_values[water_values.water_quality==i].target.value_counts(), '\n')
        
print('Quantity')
for i in water_values.quantity.unique(): 
    if 'unknown' in i: 
        print(water_values[water_values.quantity==i].target.value_counts(), '\n')
        
print('Payment')
for i in water_values.payment.unique(): 
    if 'unknown' in i: 
        print(water_values[water_values.payment==i].target.value_counts(), '\n')

Source
functional                 32
non functional             30
functional needs repair     4
Name: target, dtype: int64 

Water_Quality
non functional             1576
functional                  264
functional needs repair      35
Name: target, dtype: int64 

Quantity
non functional             561
functional                 213
functional needs repair     14
Name: target, dtype: int64 

Payment
non functional             4193
functional                 3524
functional needs repair     432
Name: target, dtype: int64 



In [3]:
#fixing dates 
median_con = water_values[water_values.construction_year != 0].construction_year.median()
water_values.construction_year.replace(0, median_con, inplace = True)
cons_year = water_values.construction_year.values 
rec_date = water_values.date_recorded.values 
def get_time_since_built(cons, rec): 
    time_array = []
    for c, r in zip(cons, rec): 
        r = int(r.split('-')[0])
        c = int(c)
        diff = r -c
        time_array.append(diff)
    return time_array

t_array = get_time_since_built(cons_year, rec_date)
water_values['time_passed'] = t_array
water_values.drop(['construction_year', 'date_recorded_date', 'year_recorded', 'date_recorded'], 
                  axis = 1, inplace = True)


#made a new column for the number of years that have passed since construction.  
#Removed the recorded date and construction date
#for cells where construction date was 0, replaced it with the median construction date of the df

In [4]:
water_values.tail(2)

,id,amount_tsh,gps_height,installer,basin,region,lga,ward,population,public_meeting,permit,extraction_type,management,payment,water_quality,quantity,source,waterpoint_type,target,time_passed
59367,31282,0.0,0,Musa,Rufiji,Dodoma,Chamwino,Mvumi Makulu,0,True,True,nira/tanira,vwc,never pay,soft,insufficient,shallow well,hand pump,functional,11
59368,26348,0.0,191,World,Wami / Ruvu,Morogoro,Morogoro Rural,Ngerengere,150,True,True,nira/tanira,vwc,pay when scheme fails,salty,enough,shallow well,hand pump,functional,9


In [5]:
#changes binary true-false to 0-1
#chanegd the colums public meeting, permit to binary instead of boolean

water_values.public_meeting.fillna(False, inplace = True)
water_values.public_meeting = water_values.public_meeting.astype(int)
water_values.permit.fillna(False, inplace = True)
water_values.permit = water_values.permit.astype(int)
water_values.tail(2)

,id,amount_tsh,gps_height,installer,basin,region,lga,ward,population,public_meeting,permit,extraction_type,management,payment,water_quality,quantity,source,waterpoint_type,target,time_passed
59367,31282,0.0,0,Musa,Rufiji,Dodoma,Chamwino,Mvumi Makulu,0,1,1,nira/tanira,vwc,never pay,soft,insufficient,shallow well,hand pump,functional,11
59368,26348,0.0,191,World,Wami / Ruvu,Morogoro,Morogoro Rural,Ngerengere,150,1,1,nira/tanira,vwc,pay when scheme fails,salty,enough,shallow well,hand pump,functional,9


In [6]:
## Combining 'other' in extract 
##combining the types of extract that have 'other' into a single type 'other'
def fix_extract_other(x): 
    if 'other' in x: 
        return 'extract_other'
    else: 
        return x

water_values['extraction_type'] = water_values.extraction_type.map(fix_extract_other)
water_values.tail()

,id,amount_tsh,gps_height,installer,basin,region,lga,ward,population,public_meeting,permit,extraction_type,management,payment,water_quality,quantity,source,waterpoint_type,target,time_passed
59364,60739,10.0,1210,CES,Pangani,Kilimanjaro,Hai,Masama Magharibi,125,1,1,gravity,water board,pay per bucket,soft,enough,spring,communal standpipe,functional,14
59365,27263,4700.0,1212,Cefa,Rufiji,Iringa,Njombe,Ikondo,56,1,1,gravity,vwc,pay annually,soft,enough,river,communal standpipe,functional,15
59366,37057,0.0,0,NaN,Rufiji,Mbeya,Mbarali,Chimala,0,1,0,swn 80,vwc,pay monthly,fluoride,enough,machine dbh,hand pump,functional,11
59367,31282,0.0,0,Musa,Rufiji,Dodoma,Chamwino,Mvumi Makulu,0,1,1,nira/tanira,vwc,never pay,soft,insufficient,shallow well,hand pump,functional,11
59368,26348,0.0,191,World,Wami / Ruvu,Morogoro,Morogoro Rural,Ngerengere,150,1,1,nira/tanira,vwc,pay when scheme fails,salty,enough,shallow well,hand pump,functional,9


In [7]:
#fixing population to replace value with mdeian population within its basin 
def fix_population_basin(df): 
    new_df = df.reset_index(drop = True)
    for idx in tqdm(range(len(new_df))): 
        pop = new_df.iloc[idx].population 
        if pop != 0: 
            continue 
        basin_value = new_df.iloc[idx].basin
        basin_median_pop = new_df[(new_df.basin == basin_value) & (new_df.population > 0)].population.median() 
        new_df.loc[idx, 'population'] = basin_median_pop
        
    return new_df     

water_values = fix_population_basin(water_values)


100%|██████████| 59369/59369 [03:10<00:00, 312.13it/s]


In [8]:
#aggregating values that have abandoned to their corresponding type

def fix_quality(x): 
    if 'abandoned' in x: 
        new_x = x.split(' ')[0]
        return new_x
    else: 
        return x

water_values.water_quality = water_values.water_quality.map(fix_quality)

In [ ]:
water_values.to_csv('../../preprocessed/WaterUpdated.csv', index = False)